# 『人を知る』人工知能講座 <br> <span style="color: #00B0F0;">Session 3 言語メディア</span> <br> <span style="background-color: #1F4E79; color: #FFFFFF;">&nbsp;3&nbsp;</span> BERTによる自然言語処理 〜日本語Fine-tuning〜 

日本語では英語のGLUEデータセットのように大規模な種々のデータセットが整備されているわけではありませんので、タグ付きコーパスからデータセットを生成し、fine-tuningします。

本演習では1日目で用いたKNBC (Kyoto University and NTT Blog Corpus)を用います。データは `/data/nlp/text/KNBC_v1.0_090925_utf8` にあります。

In [ ]:
!ls -l /data/nlp/text/KNBC_v1.0_090925_utf8

1日目と同様、アノテーションを可視化できる html を確認してみましょう。

In [ ]:
from IPython.display import HTML

KNBC_dir = "/data/nlp/text/KNBC_v1.0_090925_utf8"
HTML("<style type='text/css'>" + open(f"{KNBC_dir}/html/knbc_article_index.css").read() + "</style>")
HTML(open(f"{KNBC_dir}/html/KN001_Kyoto_1-1-17-01.html").read())

このコーパスを使って、以下のタスクをfine-tuningで解きます。

* 評判分析
* 文書分類
* 固有表現解析

## 1. 評判分析

GLUEの最初で見た SST-2 データセットと全く同じ形式のファイルを生成し、fine-tuningのコマンドを動かします。

再掲: KNBCでは「批評」(賛成と反対)や「感情」(好きと嫌い)など、いくつかの軸について、評判を保持している人や評判の対象などが付与されています。ここではfine-tuningを動かすことを目的としますので、かなり荒っぽいですが、「批評」の軸について賛成(「批評＋」と表記されている)を表す表現があれば文全体をpositive、反対(同様に「批評−」)があれば文全体をnegativeとみなすことにします。

### 手順1. 前処理

上記の基準をもとに以下のpythonスクリプトで評判分析のデータセットを生成します。1日目と同じスクリプトです。

In [ ]:
import glob
import os
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def get_words(sid):
    # sid: KN001_Keitai_1-1-12-01
    # -> KNBC_v1.0_090925/corpus1/KN001_Keitai_1/KN001_Keitai_1-1-12-01
    subdirname = (sid.split("-"))[0]
    filename = f"{KNBC_dir}/corpus1/{subdirname}/{sid}"
    if os.path.exists(filename) is False:
        return None

    # 単語集合を得る (正解単語区切を利用)
    words = []
    with open(filename, "r", encoding="utf-8") as reader:
        buf = ""
        for line in reader.readlines():
            if line.startswith(("#", "*", "+", "EOS")):
                continue
            word = (line.split(" "))[0]
            words.append(word)

    return words

def get_sentiment_label(sentiment_type):
    # 批評＋を含めば1, 批評−を含めば0
    # 両方含むものはスキップ
    if "批評＋" in sentiment_type and "批評−" in sentiment_type:
        return None
    if "批評＋" in sentiment_type:
        return 1
    if "批評−" in sentiment_type:
        return 0
    return None

out_dir = "sentiment_analysis_KNBC"
os.makedirs(out_dir, exist_ok=True)
f_out = open(f"{out_dir}/all.tsv", "w")

for filename in glob.glob(f"{KNBC_dir}/corpus2/*"):
    with open(filename, "r", encoding="utf-8") as reader:
        buf = ""
        for line in reader.readlines():
            # 文ID, 文, 評判保持者, 評判表現, 評判タイプ, 評判対象
            # KN001_Keitai_1-1-12-01	確かにプリペイドには、いくつかの弱点がある。	[著者]	いくつかの弱点がある	批評−	プリペイド
            sid, sentence, _, _, sentiment_type, _ = line.strip("\n").split("\t")
            words = get_words(sid)
            if words is None:
              # logger.warning(f"skip: {sid} {sentence}")
              continue
            label = get_sentiment_label(sentiment_type)
            tokenized_sentence = " ".join(words)
            if label is not None:         
                print(f"{tokenized_sentence}\t{label}", file=f_out)

f_out.close()

print("完了!")
!wc -l sentiment_analysis_KNBC/*

680文生成されました。

データの中身を見てみます。以下ではランダムに選んだ10件を表形式で表示しています。

In [ ]:
import pandas as pd

data_sentiment = pd.read_csv('sentiment_analysis_KNBC/all.tsv', encoding='utf-8', delimiter='\t', names=('sentence', 'label'))

data_sentiment.sample(10).style.set_table_styles(
                [{'selector': 'th',
                  'props': [('text-align', 'center')]}, 
                 {'selector': 'td',
                  'props': [('text-align', 'left')]}])

pandasを使えば以下のようにラベルの頻度を簡単に集計することができます。"1"が"0"の2倍弱多いことがわかります。

In [ ]:
data_sentiment["label"].value_counts()

次にtrain/dev/testに分割 (split)します。ここでは8:1:1の割合で分割します。ここでもpandasを使います。

In [ ]:
 import numpy as np
 
 def split_train_dev_test(data, dirname, header,
                         split_ratios=(0.8, 0.1, 0.1)):

    ts = data.shape
    df = pd.DataFrame(data)
    shuffle_df = df.reindex(np.random.permutation(df.index))

    indice_1 = int(ts[0] * split_ratios[0])
    indice_2 = int(ts[0] * (split_ratios[0] + split_ratios[1]))

    shuffle_df[:indice_1].to_csv(f"{dirname}/train.tsv",header=header, sep='\t', index=False)
    shuffle_df[indice_1:indice_2].to_csv(f"{dirname}/dev.tsv",header=header, sep='\t', index=False)
    shuffle_df[indice_2:].to_csv(f"{dirname}/test.tsv",header=header, sep='\t', index=False)        

In [ ]:
split_train_dev_test(data_sentiment, "sentiment_analysis_KNBC", ("sentence", "label"))
print("完了!")

以下のように分割されました。

In [ ]:
!wc -l sentiment_analysis_KNBC/*.tsv

中身をみてみます。SST-2と同じ形式のファイルができました。

In [ ]:
!head sentiment_analysis_KNBC/train.tsv

### 手順2. Fine-tuning

英語のGLUEのfine-tuningと同様に run_glue.py を実行します。変更点は以下のとおりです。
*   --model_name_or_pathオプションで日本語pre-trainedモデルを指定 (/data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers)
*   --do_lower_caseを削除 (tokenizer_config.jsonでfalseにしていますが念のため)
*   --data_dirオプションで先ほど作ったデータのディレクトリを指定

SST-2よりもさらにサイズが小さいため、3エポックでも2分程度で終わります。


In [ ]:
!python ./transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name "SST-2" \
    --do_train \
    --do_eval \
    --save_steps 1000 \
    --data_dir sentiment_analysis_KNBC \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir KNBC_result/sentiment_analysis/ \
    --overwrite_output_dir \
    --overwrite_cache

一番最後の数字がdevにおける精度 (accuracy)で、80%前後になると思います。1日目のLSTMによるモデルでは精度70%前後でしたので、BERTがいかに強力であるかがおわかりかと思います。

一般に、ニューラルネットワークのモデルで高い精度を達成するために大量のトレーニングデータが必要と言われます。しかし、BERTはBi-LSTMのようなこれまでのニューラルネットワークに比べるとそれほどトレーニングデータを必要としません。これは転移学習 (transfer learning)のおかげです。

### 手順3. 予測

次に、好きな文を入力し、予測 (prediction)してみましょう。まず、上記で学習したモデルを別のディレクトリにコピーしておきましょう。

In [ ]:
!mkdir -p my_result
!cp -pr KNBC_result/sentiment_analysis/ my_result/

上記で使用したデータも別のディレクトリにコピーしましょう。

In [ ]:
!cp -r sentiment_analysis_KNBC my_sentiment_analysis

そして、以下のようにしてdev.tsvを好きな文で上書きしてください。ここではわかち書きは手動で行ってください。ラベルは使いませんが0にしておいてください。

In [ ]:
!echo -e 'sentence\tlabel\nパフェ は 大変 美味しかった 。\t0' > my_sentiment_analysis/dev.tsv
!head my_sentiment_analysis/dev.tsv

以下のコマンドで予測します。先ほどとの違いは以下です。
* --do_train オプションを削除
* --save_steps 1000を削除
* --data_dir,  --output_dirオプションで今作ったディレクトリを指定

In [ ]:
!python ./transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name "SST-2" \
    --do_eval \
    --save_steps 1000 \
    --data_dir my_sentiment_analysis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --output_dir my_result/sentiment_analysis/ \
    --overwrite_output_dir \
    --overwrite_cache
!cat my_result/sentiment_analysis/dev_predictions.txt

結果が一番下に出ます。いかがでしたでしょうか。

## 2. 文書分類

次に文書分類タスクを行います。先ほどの評判分析は1か0の2ラベルでしたが、今度は4カテゴリです。また、入力は1文ではなく文章になりますが、単に全部連結したものを入力とし、1文と同様に扱います。

### 手順1. 前処理

In [ ]:
out_dir = "text_classification_KNBC"
os.makedirs(out_dir, exist_ok=True)
labels = [ "Keitai", "Kyoto", "Gourmet", "Sports"]

def get_label(basename):
    for label in labels:
        if label in basename:
            return label
    else:
        return None

f_out = open(f"{out_dir}/all.tsv", "w")

for dir in glob.glob(f"{KNBC_dir}/corpus1/*"):
    basename = os.path.basename(dir)
    words = []
    
    # for each sentence
    sentence_index = 1
    while (True):
        filename = f"{dir}/{basename}-1-{sentence_index}-01"
        if os.path.exists(filename) is False:
            break

        header = True
        with open(filename, "r", encoding="utf-8") as reader:
            buf = ""
            for line in reader.readlines():
                if line.startswith(("#", "*", "+", "EOS")):
                    continue
                word = (line.split(" "))[0]
                
                # skip: category (例: [京都観光])
                if sentence_index == 1 and header is True:
                    continue
                if header is True and word == "]":
                    header = False
                words.append(word)
        sentence_index += 1

    label = get_label(basename)
    if label is not None:
        print("{}\t{}".format(" ".join(words), label), file=f_out)
    
f_out.close()
print("完了!")

しばらくは評判分析と同様のコマンドですので、どんどんコマンドを実行しながら内容を確認してください。

In [ ]:
import pandas as pd
data_text_classification = pd.read_csv('text_classification_KNBC/all.tsv', encoding='utf-8', delimiter='\t', names=('sentence', 'label'))
data_text_classification.sample(10).style.set_table_styles(
                [{'selector': 'th',
                  'props': [('text-align', 'center')]}, 
                 {'selector': 'td',
                  'props': [('text-align', 'left')]}])

In [ ]:
data_text_classification["label"].value_counts()

In [ ]:
split_train_dev_test(data_text_classification, "text_classification_KNBC", ("sentence", "label"))
print("完了!")

In [ ]:
!wc -l text_classification_KNBC/*.tsv

ここまでは評判分析と全く同じです。

### 練習問題 1
ラベルが変わったことによりデータを読みこむ部分を追加しなければいけません。データの読みこみは transformers/transformers/data/processors/glue.py で行っています。このファイルを編集するにはファイル一覧の画面に移動して、ファイル名をクリックするとファイルを開くことができます。

SST-2は以下のクラス Sst2Processor でデータの読みこみをしています。これを真似して TextClassificationKNBCProcessor を作ってみましょう。317行目あたりにSst2Processorをコピーして TextClassificationKNBCProcessor を作っていますので、関数 get_labels のところだけを変更してください。

あとは同じファイルの末尾に、ラベルの数、どのプロセッサを使うのか、タスクが分類(classification)か回帰(regression)であるかを指定するところがあります。今回は時間の都合上、すでに以下のように追加しています。タスク名は tc-knbc としています(tcはtext classificationの意味)。

ファイルをsaveしてください。

あと、transformers/transformers/data/metrics/\_\_init\_\_.py で何を評価尺度として用いるかを指定しているところがあります。これも以下のようにすでに追加しています。

transformers/transformers以下のファイルを更新した場合はpipで再インストールする必要がありますので以下を実行してください。

In [ ]:
!pip install transformers/

### 手順2. Fine-tuning

準備が整ったのでfine-tuningしましょう。--task_nameで"tc-knbc"を指定し、--output_dirで KNBC_result/text_classification/ を指定します。

In [ ]:
!python ./transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name "tc-knbc" \
    --do_train \
    --do_eval \
    --save_steps 1000 \
    --data_dir text_classification_KNBC \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir KNBC_result/text_classification/ \
    --overwrite_output_dir \
    --overwrite_cache

3エポック回して約1分で終わります。4カテゴリが結構はっきり異なるジャンルなので90%前後と、高い精度で分類できました。

## 3. 固有表現抽出

最後に固有表現抽出 (Named Entity Recognition, NER) を行います。固有表現抽出とはテキスト中の人名、地名、組織名などの固有表現 (Named Entity)を抽出するタスクです。

これまでの講義でも説明されたとおり、固有表現抽出は系列ラベリングと呼ばれる手法で解かれることが多いです。以下の例で説明します。以下の文では「太郎」がPERSON、「京都大学」がORGANIZATIONです。「太郎」は1形態素ですが、「京都大学」は2形態素からなります。形態素が固有表現の場合、ラベルの頭にB (Begin)またはI (Inside)を付与し、固有表現でない場合、O (Outside)とし、各形態素のラベルを推定する問題となります。

単語 | 固有表現ラベル
--- | ---
太郎 | B-PERSON
は | O
京都 | B-ORGANIZATION
大学 | I-ORGANIZATION
に | O
行った | O

日本語の固有表現解析ではIREX (Information Retrieve and Extraction Exercise)で定義された、組織名(ORGANIZATION), 人名(PERSON), 地名(LOCATION), 固有物名(ARTIFACT), 日付表現(DATE), 時間表現(TIME), 金額表現(MONEY), 割合表現(PERCENT)の8種類を対象とすることが多いです。

### 手順1. 前処理

以下のpythonスクリプトで固有表現データセットを生成します。

In [ ]:
import glob
import os
import logging
import re
import random

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

random.seed(1)

corpus_types = [ "train", "dev", "test" ]
os.makedirs("ner_KNBC", exist_ok=True)
fs_out = { corpus_type: open(f"ner_KNBC/{corpus_type}.txt", "w") for corpus_type in corpus_types }

ne_pat = re.compile(r"<NE:(.+?):(.+?)>")

def get_corpus_type():
    # train:dev:test = 8:1:1 に split する
    rand = random.random()
    
    if rand >= 0.2:
        return "train"
    elif 0.1 <= rand < 0.2:
        return "dev"
    else:
        return "test"

corpus1_dirname = f"{KNBC_dir}/corpus1"

doc_num = 0
for dir in glob.glob(f"{corpus1_dirname}/*"):
    basename = os.path.basename(dir)
    corpus_type = get_corpus_type()

    f_out = fs_out[corpus_type]

    # for each sentence
    sentence_index = 1
    while (True):
        filename = f"{dir}/{basename}-1-{sentence_index}-01"
        if os.path.exists(filename) is False:
            break

        words, ne_labels = [], []
        with open(filename, "r", encoding="utf-8") as reader:
            buf = ""
            for line in reader.readlines():
                if line.startswith(("#", "*", "+", "EOS")):
                    continue
                word = (line.split(" "))[0]
                words.append(word)

                # 例：黒田 くろだ 黒田 名詞 6 人名 5 * 0 * 0 "疑似代表表記 代表表記:黒田/くろた" <疑似代表表記>..<NE:PERSON:head>
                m = ne_pat.search(line)
                if m:
                    category, position = m.groups()
                    if category == "OPTIONAL":
                        label = "O"
                    else:
                        if position == "head" or position == "single":
                            position_label = "B"
                        else:
                            position_label = "I"
                        label = f"{position_label}-{category}"
                else:
                    label = "O"
                ne_labels.append(label)

            for word, ne_label in zip(words, ne_labels):
                print(f"{word} {ne_label}", file=f_out)
            print(file=f_out)

        sentence_index += 1

for corpus_type in corpus_types:
    fs_out[corpus_type].close()

print("完了!")
!wc -l ner_KNBC/*

中身を見てみます。1カラム目が見出し、2カラム目が固有表現ラベルになっていて、空行が文区切りを表わします。

In [ ]:
!head -n 50 ner_KNBC/dev.txt

### 練習問題 2
fine-tuningの前に、ラベル一覧を得ておく必要があります。1行に1ラベルとし、ner_KNBC/labels.txtに保存してください。

In [ ]:
########## ここにコマンドを書いて下さい
!cat ner_KNBC/train.txt | grep -v "^$" | cut -d " " -f 2 | sort | uniq > ner_KNBC/labels.txt
!cat ner_KNBC/labels.txt
##########

### 手順2. Fine-tuning

系列ラベリングのfine-tuningは run_ner.py というスクリプトを使います。オプションはこれまでとほぼ同様です。--labelsオプションで上で生成したlabel一覧のファイルを指定するくらいが異なることです。

In [ ]:
!python ./transformers/examples/run_ner.py --data_dir ./ner_KNBC/ \
--model_type bert \
--labels ./ner_KNBC/labels.txt \
--model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
--output_dir KNBC_result/ner \
--max_seq_length 128 \
--num_train_epochs 3 \
--per_gpu_train_batch_size 16 \
--save_steps 1000 \
--seed 1 \
--do_train \
--do_eval \
--do_predict \
--overwrite_output_dir \
--overwrite_cache

3エポックで10分弱かかります。この間に、GPU使用状況を確認できる nvidia-smi コマンドを使ってみましょう。
2日目にJuman++を使った時と同じように、New → Terminal で Terminal を開き、 `nvidia-smi` と打ってみてください。
重要なのは以下です。
* Volatile GPU-Util: GPUがどれくらい使われているか。100%に近いほどよい
* GPU Memory Usage: GPUメモリがどれくらい使われているか。他に制約がなければ最大に近いまで使うとよい

しばらく実行し続けるときは例えば `nvidia-smi -l 3` と打つと3秒ごとに実行されます。

最後に出ている数字が test での精度です。F値で 0.75 程度です。

システムの出力を簡単に確認してみましょう。KNBC_result/ner/test_predictions.txt がシステムの出力です。

In [ ]:
!head -20 KNBC_result/ner/test_predictions.txt

### 練習問題 3*
上記の run_ner.py のオプション --per_gpu_train_batch_size はトレーニング時のバッチサイズを指定するものです。これは128や4にして実行してみてください。

### 手順3. 結果の可視化

システムの出力を検討する上で結果をわかりやすく表示することは非常に重要です。ここでは spacy というライブラリが提供している displacy を使って、固有表現解析の結果を可視化してみます。spacy はpipで簡単にインストールすることができます。(以下で使うtermcolorというライラブリも一緒にインストールしておきます)

In [ ]:
!pip install spacy termcolor

displacy では以下のように文 (text)と固有表現の集合 (ents)を与えることによって、可視化することができます。「ents」のそれぞれの固有表現のstart/endはそれぞれ文頭からの文字数を表しています。(endは固有表現の末尾の文字位置 + 1)

In [ ]:
import spacy
from spacy import displacy

ex = [{"text": "太郎 は 京都 大学 に 通っている 。",
       "ents": [{"start": 0, "end": 2, "label": "PERSON"},
                {"start": 5, "end": 10, "label": "ORG"}]}
      ]
displacy.render(ex, style="ent", manual=True, jupyter=True)

KNBCはコーパスサイズが小さかったためF値が0.75程度でしたが、固有表現解析で標準的に用いられているCRL固有表現データ(約1万文)ではF値0.92程度になります。古典的機械学習手法を用いた[笹野ら08]ではF値0.89と報告されていますので、固有表現解析でもBERTが強力であることがわかります。あらかじめ学習を走らせておき、その結果を /data/nlp/tool/bert/CRL 以下においていますので、これを用いて結果を可視化してみましょう。
**(これは毎日新聞のデータですので、持ち出さないようにお願いします)**

以下のpythonコードでシステムの出力と正解を可視化します。文単位でシステムの出力と正解が一致する場合はシステムの出力(=正解)を表示し、1文のどこかが異なる場合は上にシステムの出力、下に正解を表示します。

In [ ]:
from seqeval.metrics.sequence_labeling import get_entities
import termcolor

tag_conversion_map = { "ORGANIZATION": "ORG",
                       "LOCATION": "LOC"}
class Word(object):
    def __init__(self, string, ner_tag, offset):
        self.string = string
        self.ner_tag = ner_tag
        self.start = offset
        self.end = offset + len(string)

def get_ner_example(lines):
    words = []
    offset = 0
    for line in lines:
        string, ner_tag = line.split(" ")
        word = Word(string, ner_tag, offset)
        words.append(word)
        # +1 は空白の分
        offset += len(word.string) + 1

    # [('PER', 0, 1), ('LOC', 3, 3)]
    entities = get_entities([ word.ner_tag for word in words ])
    spacy_entities = []
    for entity in entities:
        ner_label, start_word_index, end_word_index = entity 
        spacy_entities.append({ "start": words[start_word_index].start,
                                                    "end": words[end_word_index].end, 
                                                    "label": tag_conversion_map[ner_label] if ner_label in tag_conversion_map else ner_label })  
        
    return { "text": " ".join([ word.string for word in words ]),
                  "ents": spacy_entities }  

def get_ner_examples(filename):
    examples = []
    with open(filename, "r", encoding="utf-8") as reader:
        lines = [] 
        for line in reader.readlines():
            line = line.rstrip("\n")
      
            if line:
                lines.append(line)
            # 空行 (文の切れ目)
            else:
                example = get_ner_example(lines)
                examples.append(example)
                lines = []

    return examples

def is_same_example(system_example, gold_example):
    assert system_example["text"] == gold_example["text"]

    # entityの数が異なる
    if len(system_example["ents"]) != len(gold_example["ents"]):
        return False
    
    for system_ent, gold_ent in zip(system_example["ents"], gold_example["ents"]):
        # start, end, labelのいずれかが異なる
        if system_ent["start"] != gold_ent["start"] or system_ent["end"] != gold_ent["end"] or system_ent["label"] != gold_ent["label"]:
            return False

    # すべて一致したので全体が一致
        return True

def display_result(system_filename, gold_filename):
    system_examples = get_ner_examples(system_filename)
    gold_examples = get_ner_examples(gold_filename)

    for i, (system_example, gold_example) in enumerate(zip(system_examples, gold_examples)):
        if i == 100:
            break
        if is_same_example(system_example, gold_example) is True:
            displacy.render(system_example, style="ent", manual=True, jupyter=True)
        else:
            print("-" * 100)
            print(termcolor.colored("system", "blue"))
            displacy.render(system_example, style="ent", manual=True, jupyter=True)
            print(termcolor.colored("gold", "red"))
            displacy.render(gold_example, style="ent", manual=True, jupyter=True)
            print("-" * 100)
      
display_result("/data/nlp/tool/bert/CRL/test_predictions.txt", "/data/nlp/tool/bert/CRL/test.txt")

### 練習問題 4
上記の結果からどのようなことがわかるか分析してみよう。

*   システムはこんな難しいものでもわかるのか
*   逆にこんなやさしいものもわからないのか
*   システムがわからなくても仕方ない難しいもの
*   システムの出力の方が正しくて正解が間違っているのではないかというもの
*   タグ付けが誤っているのではないか


以上のように、BERTによるfine-tuningではプログラムの中身をほぼ変更することなく、入力データを用意するだけでなく、様々なタスクのfine-tuningを行うことができます。